[Node 69: Web-Framework Django](http://www-static.etp.physik.uni-muenchen.de/kurs/Computing/python2/node69.html)

Navigation:

**Next:** [Aufgaben](node70.ipynb) **Up:** [Aufgaben](node70.ipynb) **Previous:** [Aufgaben](node70.ipynb)

## Web Framework Django

Web frameworks typically combine the following components:
* integrated web server
* Templating system (text or XML based)
* Connection to a database

Django is what is known as a "lightweight" web framework. A Django website consists of a Django project, which usually consists of several applications. An application consists of 3 components:
* Models: object-oriented representation of the application's persistent data
* Views: Controllers that query models and generate data using templates. Templates determine the appearance of the website.
* URL mappings: Mapping of partial URLs to individual views

![Image architecture](figures/architecture.png "Image architecture")

### Model template view
* <font color=#0000ff> **Models :**</font> What things are
* <font color=#0000ff> **Views :**</font> How things are processed
* <font color=#0000ff> **Templates :**</font> How things are presented

### A wiki service in Django

As an example for a Django application we create a simple [Wiki-Service](https://de.wikipedia.org/wiki/Wiki) in the following step by step. As the steps build on each other, there is a [Wiki-Service](https://de.wikipedia.org/wiki/Wiki) representing the following steps.

We are using Django version 2.2.10, this can be installed with
```bash
python3 -m pip install Django==2.2.10
```

We start by setting up a new Django project. (Django must be installed for this.)
```bash
cd $HOME
mkdir djangoprojs
cd djangoprojs
django-admin startproject wikicamp
```

Several files are generated:
```bash
laptop:~/djangoprojs$ cd wikicamp/
laptop:~/djangoprojs/wikicamp$ ls
manage.py wikicamp
```

(At this point it would be [git-commit 713471c](https://github.com/fuenfundachtzig/PythonkursDjango/commit/713471c4292c035970a7c8b2a794cc70ca34e672).)

To create the database, we use
```bash
python3 manage.py migrate
```

Start the development server on port 9090 with:
```bash
python3 manage.py runserver 9090
```
and open the following address in a web browser: http://localhost:9090/

---
Not much is happening there, you first have to create the application templates with:
```bash
laptop:~/djangoprojs/wikicamp$ python3 manage.py startapp wiki
laptop:~/djangoprojs/wikicamp$ ls
db.sqlite3  manage.py  wiki  wikicamp
```

(git: commit 2)

Edit <font color=#0000e6>wikicamp/settings.py</font> and change the following lines - this is how we define the applications or Django modules used:
```python
....
INSTALLED_APPS = (
    ...
    'wiki',
....
TIME_ZONE = 'Europe/Berlin'
....
)
```

Edit the file <font color=#0000e6>wiki/models.py</font> :
```python
from django.db import models
# Create your models here.                                                      
class Page(models.Model):
    name = models.CharField(max_length=20, primary_key=True)
    content = models.TextField(blank=True)
```

Run the following commands:
```bash
python3 manage.py makemigrations
python3 manage.py migrate
```

Restart the Django server:
```bash
laptop:~/djangoprojs/wikicamp$ python3 manage.py runserver 9090
```

Open the <font color=#0000e6>wikicamp/urls.py</font> file and define a URL image:
```python
...
    from django.urls import re_path
    from wiki import views
...
    path('admin/', admin.site.urls),
    re_path(r'^wikicamp/(?P<page_name>[^/]+)/$', views.view_page)
```

---
We now create the "view" by editing the file <font color=#0000e6>wiki/views.py</font>:
```python
# Create your views here.
from wiki.models import Page
from django.shortcuts import render_to_response
from django.template.context_processors import csrf # https://docs.djangoproject.com/en/3.0/ref/csrf/
def view_page(request, page_name):
    try:
        page = Page.objects.get(pk=page_name)
        # need to add response later        
    except Page.DoesNotExist:
        c = {"page_name": page_name}
        c.update(csrf(request))
        return render_to_response("create.html", c)
```

Calling the URL <font color=#0000e6>http://localhost:9090/wikicamp/Start/</font> again returns a new error: <font color=#ff0000> **'TemplateDoesNotExist at /wikicamp/Start/ create.html'**</font>

We now create this template. Create the <font color=#0000e6>djangoprojs/wikicamp/templates</font> subdirectory and add the <tt>templates</tt> directory to <tt>DIRS</tt> in <font color=#0000e6> wikicamp/settings.py</font> enter:
```python
TEMPLATES = [
    {
        'BACKEND': 'django.template.backends.django.DjangoTemplates',
        'DIRS': ['templates'],
)
```

Create the file <font color=#0000e6>templates/create.html</font> with the following content:
```html
  <head>
    <title>{{page_name}} - Create</title>
  </head>
  <body>
    <h1>{{page_name}}</h1>
    This page does not exist. <a href="/wikicamp/{{page_name}}/edit/">Create?</a>
  </body>
```

(git: commit 3)

---

Calling the URL <font color=#0000e6>http://localhost:9090/wikicamp/Start/</font> again now returns a correct HTML page. When the <font color=#008000> *Create*</font> link is clicked, an error message about the missing page (404) is received.

We now add this to <font color=#0000e6>wiki/views.py</font>:
```python
def edit_page(request, page_name):
    try:
        page = Page.objects.get(pk=page_name)
        content	= page.content
    except Page.DoesNotExist:
        content = ""
    c = {"page_name": page_name, "content":content}   
    c.update(csrf(request))
    return render_to_response("edit.html", c)
```

and in `wikicamp/urls.py`:
```python
    re_path(r'^wikicamp/(?P<page_name>[^/]+)/edit/$', views.edit_page),
    re_path(r'^wikicamp/(?P<page_name>[^/]+)/save/$', views.save_page),
```

This initially generates errors for the web server:
<font color=#ff0000>
**AttributeError: module 'wiki.views' has no attribute 'save_page'**
</font>

To save the text field content, we now define the <font color=#008000> *save_page()*</font> method in <font color=#0000e6>wiki/views.py</font>
```python
....
from django.http import HttpResponseRedirect
def save_page(request, page_name):
    content = request.POST["content"]
    try:
        page = Page.objects.get(pk=page_name)
        page.content = content
    except Page.DoesNotExist:
        page = Page(name=page_name, content=content)
    page.save()
    return HttpResponseRedirect("/wikicamp/"+ page_name + "/")
```



Calling <font color=#0000e6>http://localhost:9090/wikicamp/Start/edit</font> again returns an error message that the template does not yet exist: <font color=#ff0000> **TemplateDoesNotExist at /wikicamp/Start/edit/**</font>
 

We now create the template <font color=#0000e6>templates/edit.html</font>:
```html
  <head>
    <title>{{page_name}} - Editing</title>
  </head>
  <body>
    <h1>Editing {{page_name}}</h1>
    <form method="post" action="/wikicamp/{{page_name}}/save/">
       {% csrf_token %}
          <textarea name="content" rows="20" cols="60">{{content}}</textarea><br/>
          <input type="submit" value="Save Page"/>
    </form>
  </body>
```

Pages can now be created but not yet displayed.
(There would be an error `ValueError: The view wiki.views.view_page didn't return an HttpResponse object. It returned None instead`.) Therefore we add the <font color=#008000> *view_page()*</font> method in the <tt>wiki/views.py</tt> file (where there is still ``# need to add response later``):
```python
...
def view_page(request, page_name):
    try:
        page = Page.objects.get(pk=page_name)
        content = page.content
        c = {"page_name": page_name, "content":content}
        c.update(csrf(request))
        return render_to_response("view.html", c)
    except Page.DoesNotExist:
        c = {"page_name": page_name}
        c.update(csrf(request))
        return render_to_response("create.html", c)
...
```

with the associated template <font color=#0000e6>templates/view.html</font> with the following content:
```html
  <head>
    <title>{{page_name}}</title>
  </head>
  <body>
    <h1>{{page_name}}</h1>
    {{content}}
    <hr/>
    <a href="/wikicamp/{{page_name}}/edit">Edit	this page?</a>
  </body>
```

(git: commit 4)

Fill in the page <font color=#008000> *Start*</font> Start with the text: <font color=#008000> *This is the start page.*</font> and click the <font color=# 008000> *Submit*</font> button. The page is saved in the database and displayed with <font color=#0000e6>templates/view.html</font>. Click the <font color=#008000> *Edit this page?*</font> link - the edit dialog will be displayed again.

We have hereby created a prototype of a wiki system with changes or creation of the following files:
<!--
* <font color=#0000e6>wiki/models.py</font>
* <font color=#0000e6>wiki/views.py</font>
* <font color=#0000e6>wiki.db</font>
* <font color=#0000e6>templates/create.html</font>
* <font color=#0000e6>templates/view.html</font>
* <font color=#0000e6>templates/edit.html</font>
* <font color=#0000e6>settings.py</font>
* <font color=#0000e6>urls.py</font>
-->
<pre>
.
└── wikicamp
├── db.sqlite3
├── manage.py
├── templates
│ ├── create.html
│ ├── edit.html
│ └── view.html
├── wiki
│ ├── admin.py
│ ├── apps.py
│ ├── migrations
│ │ ├── ...
│ ├── models.py
│ ├── tests.py
│ ├── views.py
└── wikicamp
├── settings.py
├── urls.py
└── wsgi.py
</pre>

#### Bonus (I): The Django admin page

Create the file <font color=#0000e6>wiki/admin.py</font> with the following content to make the <font color=#0000e6>Page</font> object visible in the future administration web page:
```python
from wiki.models import Page
from django.contrib import admin
admin.site.register(Page)
```

An administrator account can be created with
```bash
python3 manage.py createsuperuser
```

Call up the administrator page in a web browser at the address http://localhost:9090/admin and log in with the username and password you just set. You have full control over your project and the data entries on the administrator page.

(Git: commit 5)

#### Bonus (II): Markdown
In order to edit the wiki page as usual with a simplified markup language, the <font color=#0000ff> **markdown**</font> module must now be inserted. Change <font color=#0000e6>wiki/views.py</font> as follows:
```python
.......
import markdown
.......
def view_page(request, page_name):
.......
    try:
        page = Page.objects.get(pk=page_name)
        content = page.content
        from django.utils.safestring import mark_safe
        c = {"page_name": page_name, "content": mark_safe(markdown.markdown(content))}
        c.update(csrf(request))
        return render_to_response("view.html", c)
.......
```

These lines make the text editable and renderable with <font color=#0000ff>[Markdown](http://de.wikipedia.org/wiki/Markdown)</font> commands.
 

You may have to install the Python module <font color=#0000ff>markdown</font>
```bash
python3 -m pip install markdown
```
 
(Git: commit 6)